In [ ]:
!pip install networkx
#!pip install mip
!pip install gurobipy

In [ ]:
import random
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
from itertools import islice
from math import ceil
import networkx as nx
import numpy as np
import time
import gurobipy as gp
from gurobipy import GRB
import math
#import mip 
#from mip import BINARY, INTEGER

In [ ]:
#seeds
seed = 9
random.seed(seed)
np.random.seed(seed)

#topology
Nodes = 4
dens = 0.5 # persentage of ones (links)

#DISTANCES 
b = np.random.random_integers(150,450,size=(Nodes,Nodes))
Mask = np.random.choice([0, 1], size=(Nodes,Nodes), p=[1-dens, dens])
c = np.multiply(b, Mask)

A = ((c + c.T)/2).astype(int)
np.fill_diagonal(A,0)

print('Adjasency matrix:\n',  A)
print('Mean length: ', np.mean(A))
print('Max length: ', np.max(A))


#G = nx.from_numpy_matrix(A)
G = nx.from_numpy_matrix(A, parallel_edges=False, create_using= nx.DiGraph)

N = list(G.nodes())#set of nodes
E = list(G.edges())#set of edges
len_e = nx.get_edge_attributes(G,'weight')# set of lengths of each edge (dict)
      
print("Edge set: ",G.edges())
print('Number of nodes: ', nx.number_of_nodes(G))
print('Number of edges: ', nx.number_of_edges(G))


pos=nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, font_weight='bold')
edge_weight = nx.get_edge_attributes(G,'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels = edge_weight)
plt.show()

In [ ]:
def clear_spectrum(G, num_slots):
    # occupation of all the channels of all the links is initially zero
    for link in G.edges:
        G.edges[link]['spectrum_slots'] = [0] * num_slots

In [ ]:
def k_shortest_paths(G, N, k):
    len_p = {}#length of each precomputed path for the demand set
    paths_dict = {}
    for src_id in N:
        for dst_id in N:
            if src_id == dst_id:
                continue        
            paths_dict[(src_id, dst_id)] = list(islice(nx.shortest_simple_paths(G, src_id, dst_id, weight='weight'), k)) 
            paths_list = paths_dict[(src_id, dst_id)]
            len_p[(src_id, dst_id)] = [nx.path_weight(G, path, weight="weight") for path in paths_list]
    return paths_dict, len_p

In [ ]:
def choose_MF(G, path, path_length, traffic):
    # {traffic_request_Gbit/s: [(maximum_reach, number_of_slots)]}
    MF_option = {200: (900, 2), 400: (600, 3)}
    ##############################################################################
    # rest of the function will be written by students
    min_slots = 1e6
    num_reg = 0
    traffic_G = 200*ceil(traffic/200)
    # choose option with lowest number of slots with reach higher than path length
    max_reach, num_slots = MF_option[traffic_G]
    if path_length <= max_reach and num_slots < min_slots:
        min_slots = num_slots
    elif path_length > max_reach and num_slots < min_slots:#regeneration 
        min_slots = ceil(path_length/max_reach)
        num_reg = ceil(path_length/max_reach)-1
    return min_slots, num_reg

In [ ]:
def First_Fit(G, path, num_slots):
    '''find the first num_slots-wide free spectrum segment along the path'''
    slots_in_band = len(G.edges[(1, 2)]['spectrum_slots'])
    for first_slot in range(slots_in_band - num_slots + 1):
        spectrum_is_free = True
        for link in zip(path, path[1:]):
            if sum(G.edges[link]['spectrum_slots'][first_slot : first_slot + num_slots]) != 0:
                spectrum_is_free = False
                break
        if spectrum_is_free:
            return first_slot
    return None

In [ ]:
def occupy_spectrum(G, path, first_slot, num_slots):
    for link in zip(path, path[1:]):
        for slot_shift in range(num_slots):
            G.edges[link]['spectrum_slots'][first_slot + slot_shift] = 1

In [ ]:
def spectrum_occupation(G, E):
    num_slots_occupied = 0
    for link in E:
        num_slots_occupied += sum(G.edges[link]['spectrum_slots'])
    return num_slots_occupied

In [ ]:
def k_shortest_path_first_fit_RSA(G, k_SP_dict, path_lengths, traffic_dict):
    chosen_paths = []
    num_reg = 0
    lost_traffic = 0
    # find route and spectrum allocation for every demand
    for (src_id, dst_id), traffic_G_list in traffic_dict.items():
        for traffic_G in traffic_G_list: # iter over traffic
            chosen_paths.append(-1)
            # sequentially check candidate paths
            for path_ind, path in enumerate(k_SP_dict[(src_id, dst_id)]):
                # choose MF that provisions traffic with lowest spectrum occupation
                path_length = path_lengths[((src_id, dst_id))][path_ind]
                num_slots, reg = choose_MF(G, path, path_length, traffic_G) 
                num_reg+=reg
                # find first available spectrum channel along this path
                first_slot = First_Fit(G, path, num_slots)
                # spectrum found
                if first_slot != None:
                    occupy_spectrum(G, path, first_slot, num_slots)
                    chosen_paths[-1] = path_ind
                    break
                else:
                    lost_traffic+=traffic_G
    return chosen_paths, num_reg, lost_traffic

In [ ]:
class DemandList(dict):
    def __setitem__(self, key, value):
        try:
            # Assumes there is a list on the key
            self[key].append(value)
        except KeyError: # If it fails, because there is no key
            super(DemandList, self).__setitem__(key, value)
        except AttributeError: # If it fails because it is not a list
            super(DemandList, self).__setitem__(key, [self[key], value])


In [ ]:
def GenTraff(demands, Nodes):
    D = DemandList() # tuple of 3 values: s, d, b <-SET OF DEMANDS
    for d in range(demands):
        s_d = random.sample(range(0, Nodes), 2)
        D[(s_d[0], s_d[1])] = np.random.choice([100,150,200,250,300,350,400])
    return D

In [ ]:
import time
#init
Capacity = 4.4*(10**3) #4.8THz
slice_width = 25# GHz
num_slots = int(Capacity/slice_width)
demands = 350 #initial number of demands
k = 4
mc = 1000

block_prob_mean = []
block_prob_confidence = []#95%
cost_mean = []
cost_confidence = []#95%
offered_traff_mean = []

 D = GenTraff(demands, Nodes)

offered_traff =sum( sum(x) for x in D.values() ) 
print('Total offered traffic [Gbps]: ', offered_traff)
clear_spectrum(G, num_slots)
        
paths_dict, len_p = k_shortest_paths(G, N, k)
SP_solution, num_reg, lost_traffic = k_shortest_path_first_fit_RSA(G, paths_dict,len_p, D)

print('Ratio of routed requests:', 1 - SP_solution.count(-1) / len(SP_solution))
print('Blocking probability:', lost_traffic / offered_traff)
print('Slots occupied:', spectrum_occupation(G, E))
print('Total cost:', 2*num_reg)
print(SP_solution)

In [ ]:
block_prob_mean